<a href="https://colab.research.google.com/github/silverstar0727/layer_thickness/blob/master/%EB%B0%95%EB%A7%89_%EB%91%90%EA%BB%98_%EC%B8%A1%EC%A0%95_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리 import

import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 처리
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import * # 일단 싹 다. 모델평가를 해야해서...
from sklearn.model_selection import * # 너도 싹 다.
import tensorflow.keras
from keras.optimizers import Adam
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
# https://github.com/4uiiurz1/keras-cosine-annealing/blob/master/cosine_annealing.py
# pytorch와 달리 tensorflow의 keras에는 cosine anneling이 구현되어 있지 않아서 직접 구현해야함
# open source활용
import math
from keras.callbacks import Callback
from keras import backend as K


class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [0]:
# Activation fucntion으로 gelu를 사용 gelu가 relu보다 효과가 좋음
# Gaussian Error Linear Units
class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'gelu': Gelu(gelu)})

In [4]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 데이터 불러오기
train = pd.read_csv('/content/gdrive/My Drive/machine_learning/박막두께/train_set.csv', index_col = 0)
test = pd.read_csv('/content/gdrive/My Drive/machine_learning/박막두께/test_set.csv', index_col = 0)
sample_submission = pd.read_csv('/content/gdrive/My Drive/machine_learning/박막두께/sample_submission.csv')

In [6]:
# 데이터 요약
print(train.describe())
print(test.describe())
print(sample_submission.describe())

             layer_1        layer_2  ...            224            225
count  170100.000000  170100.000000  ...  170100.000000  170100.000000
mean      155.043269     155.010053  ...       0.631455       0.633718
std        86.527234      86.602764  ...       0.195622       0.195101
min        10.000000      10.000000  ...      -0.005321      -0.005659
25%        80.000000      80.000000  ...       0.508407       0.511000
50%       160.000000     160.000000  ...       0.677139       0.679399
75%       230.000000     230.000000  ...       0.787409       0.789263
max       300.000000     300.000000  ...       0.941404       0.943648

[8 rows x 230 columns]
                  0             1  ...           224           225
count  18900.000000  18900.000000  ...  18900.000000  18900.000000
mean       0.297972      0.298041  ...      0.632419      0.634869
std        0.182991      0.183320  ...      0.194512      0.194490
min       -0.013451     -0.013265  ...     -0.001584     -0.005085
25

In [0]:
# 슬라이싱을 통한 변수분리
## 자료형은 pandas가 아닌 numpy
train_X = np.array(train.iloc[:, 4:]) 
train_Y = np.array(train.iloc[:,0:4])
test_X = np.array(test.iloc[:, 0:]) 

In [0]:
# 모델을 평가하기 위한 데이터분리 (비율은 10%)
# random_state는 튜닝가능한 하이퍼 파라미터?

x_train, x_eval, y_train, y_eval = train_test_split(train_X, train_Y, test_size = 0.1, random_state = 52)

In [13]:
drop_ratio = 0.00001

model = Sequential()
model.add(Dense(2048, input_dim = len(x_train[0])))
model.add(BatchNormalization())
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))

model.add(Dense(2040))
model.add(BatchNormalization())
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))

model.add(Dense(1020))
model.add(BatchNormalization())
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))

model.add(Dense(510))
model.add(BatchNormalization())
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))

model.add(Flatten())
model.add(Dense(units=4))
model.add(Activation(gelu))
model.add(Dropout(drop_ratio))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 2017)              457859    
_________________________________________________________________
batch_normalization_6 (Batch (None, 2017)              8068      
_________________________________________________________________
activation_7 (Activation)    (None, 2017)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 2017)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 2013)              4062234   
_________________________________________________________________
batch_normalization_7 (Batch (None, 2013)              8052      
_________________________________________________________________
activation_8 (Activation)    (None, 2013)             

In [0]:
lr = 3e-4
lr_d = 0.0
patience = 200

n_fold = 13
k_fold = KFold(n_splits = n_fold, shuffle = True, random_state = 1000).split(x_train, y_train)  

for k, (train, test) in enumerate(k_fold):
  

SyntaxError: ignored

In [0]:
def model_fit(x_train, y_train):
  drop_ratio = 0.00001

  model = Sequential()
  model.add(Dense(2048, input_dim = len(x_train[0])))
  model.add(BatchNormalization())
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))

  model.add(Dense(2040))
  model.add(BatchNormalization())
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))

  model.add(Dense(1020))
  model.add(BatchNormalization())
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))

  model.add(Dense(512))
  model.add(BatchNormalization())
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))

  model.add(Dense(510))
  model.add(BatchNormalization())
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))

  model.add(Flatten())
  model.add(Dense(units=4))
  model.add(Activation(gelu))
  model.add(Dropout(drop_ratio))
  model.summary()

  model.compile(loss='mae', optimizer= 'nadam', metrics=['mae'])

  model.fit(x_train, y_train, batch_size = 4096, epochs = 1000, validation_split = 0.05, verbose = 1)

  return model

def pred_ensemble(models, x_test):
  yhats = [model.predict(x_test) for model in models]
  yhats = array(yhats)

  result = mean(yhats)
  return result

for i in range(n):
  models = [model_fit(x_train, y_train) for i in range()]

In [0]:
model.compile(loss='mae', optimizer= 'nadam', metrics=['mae'])

In [15]:
model.fit(x_train, y_train, batch_size = 4096, epochs = 1000, validation_split = 0.05, verbose = 1)

Epoch 1/1000
36/36 [==============================] - 7s 186ms/step - loss: 150.3918 - mae: 150.3918 - val_loss: 150.3146 - val_mae: 150.3146
Epoch 2/1000
36/36 [==============================] - 6s 176ms/step - loss: 143.8157 - mae: 143.8157 - val_loss: 127.9220 - val_mae: 127.9220
Epoch 3/1000
36/36 [==============================] - 6s 177ms/step - loss: 134.8311 - mae: 134.8311 - val_loss: 133.8797 - val_mae: 133.8797
Epoch 4/1000
36/36 [==============================] - 6s 177ms/step - loss: 123.0807 - mae: 123.0807 - val_loss: 142.9989 - val_mae: 142.9989
Epoch 5/1000
36/36 [==============================] - 6s 176ms/step - loss: 109.0844 - mae: 109.0844 - val_loss: 138.3094 - val_mae: 138.3094
Epoch 6/1000
36/36 [==============================] - 6s 176ms/step - loss: 94.7912 - mae: 94.7912 - val_loss: 135.9320 - val_mae: 135.9320
Epoch 7/1000
36/36 [==============================] - 6s 176ms/step - loss: 80.7809 - mae: 80.7809 - val_loss: 146.5709 - val_mae: 146.5709
Epoch 8/10

In [16]:
model.evaluate(x_eval, y_eval)

532/532 [==============================] - 3s 5ms/step - loss: 0.7207 - mae: 0.7207


[0.7206838726997375, 0.7206838726997375]

In [0]:
pred = model.predict(x_eval)

In [0]:
pred_test = model.predict(test_X)